In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup
import warnings

# Ignore any warnings
warnings.filterwarnings("ignore")

# Load datasets
accident_location = pd.read_csv('ACCIDENT/ACCIDENT_LOCATION.csv')
accident_details = pd.read_csv('ACCIDENT/ACCIDENT.csv')
precise_location = pd.read_csv('ACCIDENT/NODE.csv')

# Rename specific columns
accident_details = accident_details.rename(columns={'Accident Type Desc': 'Accident Type'})
precise_location = precise_location.rename(columns={'LGA_NAME': 'Name'})

# Merge datasets
accident_data_merged = pd.merge(accident_details, accident_location, on='ACCIDENT_NO')
accident_data_merged = pd.merge(accident_data_merged, precise_location, on='ACCIDENT_NO')

# Filter by speed zone
accident_data_merged = accident_data_merged[accident_data_merged['SPEED_ZONE'] <= 110]

# Remove unnessary columns
relevant_columns = ['ACCIDENTDATE', 'ROAD_NAME', 'ROAD_TYPE', 'Accident Type', 'SPEED_ZONE', 'Name', 'Lat', 'Long', 'ACCIDENT_NO']
accident_data_merged = accident_data_merged[relevant_columns]

# Convert to GeoDataFrame
victoria_geoframe = gpd.GeoDataFrame(accident_data_merged, geometry=gpd.points_from_xy(accident_data_merged.Long, accident_data_merged.Lat))

# Create map centered around Melbourne
interactive_map = folium.Map(location=[-37.8136, 144.9631], zoom_start=7)

# Create a parent group for all years
span_years = folium.FeatureGroup(name='Display Selected').add_to(interactive_map)

# Extract unique years for the dropdown menu
selectable_year = sorted(accident_data_merged['ACCIDENTDATE'].str.extract(r'(\d{4})')[0].unique())

# Plot data year by year
for year in selectable_year:
    year_data = victoria_geoframe[victoria_geoframe['ACCIDENTDATE'].str.contains(str(year))]
    group = FeatureGroupSubGroup(span_years, name=str(year)).add_to(interactive_map)
    datapoint = MarkerCluster().add_to(group)
    
    for idx, row in year_data.iterrows():
        popup_text = f"Date: {row['ACCIDENTDATE']}<br>Location: {row['ROAD_NAME']}, {row['ROAD_TYPE']}<br>Accident Type: {row['Accident Type']}<br>Speed Zone: {row['SPEED_ZONE']}<br>Name: {row['Name']}"
        folium.Marker([row['Lat'], row['Long']], popup=popup_text).add_to(datapoint)

folium.LayerControl(collapsed=False).add_to(interactive_map)

interactive_map.save("victoria_crash_analysis.html")

interactive_map